In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import os
import warnings

warnings.filterwarnings('ignore')

In [0]:
# set random seed
tf.random.set_seed(42)
np.random.seed(42)

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [7]:
x_train.shape

(60000, 28, 28)

In [8]:
y_train.shape

(60000,)

In [0]:
@tf.function
def _reshape(x):
  return tf.expand_dims(x, axis=2)

@tf.function
def casting(x1, x2):
  return tf.cast(x1, tf.float32), tf.cast(x2, tf.float32)

@tf.function
def normalize(x):
  return x / 255.

In [0]:
# Train Data
# x
train_data_x = tf.data.Dataset.from_tensor_slices(x_train.astype('float'))
train_data_x = train_data_x.map(normalize)
train_data_x = train_data_x.map(_reshape)
# y 
train_data_y = tf.data.Dataset.from_tensor_slices(y_train)
# zip
train_data = tf.data.Dataset.zip((train_data_x, train_data_y))
train_data = train_data.map(casting)
train_data = train_data.shuffle(20000)
train_data = train_data.batch(128)
train_data = train_data.prefetch(36)

# Test Data
# x
test_data_x = tf.data.Dataset.from_tensor_slices(x_test.astype('float'))
test_data_x = test_data_x.map(normalize)
test_data_x = test_data_x.map(_reshape)
# y 
test_data_y = tf.data.Dataset.from_tensor_slices(y_test)
# zip
test_data = tf.data.Dataset.zip((test_data_x, test_data_y))
test_data = test_data.map(casting)
test_data = test_data.batch(128)
test_data = test_data.prefetch(36)

In [54]:
train_data

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.float32)>

In [0]:
class Residual_Block(tf.keras.layers.Layer):
  def __init__(self, filters, kernel_size, **kwargs):
    super(Residual_Block, self).__init__(**kwargs)
    self.filters = filters
    self.kernel_size = kernel_size
    self.conv1 = tf.keras.layers.Conv2D(self.filters, self.kernel_size, strides=1, padding='same')
    self.conv2 = tf.keras.layers.Conv2D(self.filters, self.kernel_size, strides=1, padding='same')
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.ac1 = tf.keras.layers.LeakyReLU()
    self.bn2 = tf.keras.layers.BatchNormalization()
    self.ac2 = tf.keras.layers.LeakyReLU()

  def call(self, x):
    z = self.conv1(x)
    z = self.ac1(z)
    z = self.bn1(z)
    z = self.conv2(z)
    z = self.ac2(z)
    z = self.bn2(z)
    z = tf.keras.layers.Add()([x, z])
    return z

In [0]:
check_input = tf.keras.layers.Input(shape=(28,28,1))
x = tf.keras.layers.Conv2D(128, 7, strides=1, padding='same')(check_input)
x = tf.keras.layers.LeakyReLU()(x)
for _ in range(10):
  x = Residual_Block(128, 4)(x)
x = tf.keras.layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.Dense(10, activation='softmax')(x)

check_model = tf.keras.models.Model(check_input, x)

In [97]:
check_model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_217 (Conv2D)          (None, 28, 28, 128)       6400      
_________________________________________________________________
leaky_re_lu_147 (LeakyReLU)  (None, 28, 28, 128)       0         
_________________________________________________________________
residual__block_64 (Residual (None, 28, 28, 128)       525568    
_________________________________________________________________
residual__block_65 (Residual (None, 28, 28, 128)       525568    
_________________________________________________________________
residual__block_66 (Residual (None, 28, 28, 128)       525568    
_________________________________________________________________
residual__block_67 (Residual (None, 28, 28, 128)       525

In [0]:
check_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [99]:
check_model.fit(train_data, epochs=10, validation_data=test_data)

Train for 469 steps, validate for 79 steps
Epoch 1/10
469/469 [==============================] - 249s 531ms/step - loss: 0.2111 - acc: 0.9380 - val_loss: 0.8384 - val_acc: 0.7836
Epoch 2/10
469/469 [==============================] - 248s 529ms/step - loss: 0.0637 - acc: 0.9807 - val_loss: 0.1041 - val_acc: 0.9686
Epoch 3/10
469/469 [==============================] - 248s 530ms/step - loss: 0.0467 - acc: 0.9857 - val_loss: 0.0903 - val_acc: 0.9708
Epoch 4/10
469/469 [==============================] - 248s 528ms/step - loss: 0.0420 - acc: 0.9870 - val_loss: 0.1529 - val_acc: 0.9582
Epoch 5/10
469/469 [==============================] - 248s 528ms/step - loss: 0.0359 - acc: 0.9888 - val_loss: 0.2122 - val_acc: 0.9373
Epoch 6/10
469/469 [==============================] - 247s 528ms/step - loss: 0.0333 - acc: 0.9898 - val_loss: 0.0505 - val_acc: 0.9844
Epoch 7/10
469/469 [==============================] - 248s 528ms/step - loss: 0.0321 - acc: 0.9897 - val_loss: 0.0832 - val_acc: 0.9773
Epoch

In [0]:
class Residual_Block(tf.keras.layers.Layer):
  def __init__(self, filters, kernel_size, **kwargs):
    super(Residual_Block, self).__init__(**kwargs)
    self.filters = filters
    self.kernel_size = kernel_size
    self.conv1 = tf.keras.layers.Conv2D(self.filters, self.kernel_size, strides=1, padding='same')
    self.conv2 = tf.keras.layers.Conv2D(self.filters, self.kernel_size, strides=1, padding='same')
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.ac1 = tf.keras.layers.LeakyReLU()
    self.bn2 = tf.keras.layers.BatchNormalization()
    self.ac2 = tf.keras.layers.LeakyReLU()

  def call(self, x):
    z = self.conv1(x)
    z = self.ac1(z)
    z = tf.nn.leaky_relu(z)
    z = self.conv2(z)
    z = tf.nn.leaky_relu(z)
    z = self.bn2(z)
    z = tf.keras.layers.Add()([x, z])
    return z

In [0]:
check_input = tf.keras.layers.Input(shape=(28,28,1))
x = tf.keras.layers.Conv2D(128, 7, strides=1, padding='same')(check_input)
x = tf.keras.layers.LeakyReLU()(x)
for _ in range(10):
  x = Residual_Block(128, 4)(x)
x = tf.keras.layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.Dense(10, activation='softmax')(x)

check_model = tf.keras.models.Model(check_input, x)

In [102]:
check_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
check_model.fit(train_data, epochs=10, validation_data=test_data)

Train for 469 steps, validate for 79 steps
Epoch 1/10
469/469 [==============================] - 238s 507ms/step - loss: 0.1814 - acc: 0.9470 - val_loss: 0.2349 - val_acc: 0.9330
Epoch 2/10
469/469 [==============================] - 245s 522ms/step - loss: 0.0534 - acc: 0.9831 - val_loss: 0.1501 - val_acc: 0.9580
Epoch 3/10
469/469 [==============================] - 244s 521ms/step - loss: 0.0413 - acc: 0.9869 - val_loss: 0.1690 - val_acc: 0.9555
Epoch 4/10
469/469 [==============================] - 245s 521ms/step - loss: 0.0328 - acc: 0.9897 - val_loss: 0.1762 - val_acc: 0.9410
Epoch 5/10
469/469 [==============================] - 245s 521ms/step - loss: 0.0294 - acc: 0.9909 - val_loss: 0.0716 - val_acc: 0.9781
Epoch 6/10
469/469 [==============================] - 244s 521ms/step - loss: 0.0274 - acc: 0.9911 - val_loss: 0.0595 - val_acc: 0.9803
Epoch 7/10
469/469 [==============================] - 245s 523ms/step - loss: 0.0244 - acc: 0.9922 - val_loss: 0.0585 - val_acc: 0.9830
Epoch

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train / 255.
x_test = x_test / 255.

x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)


In [0]:
check_input = tf.keras.layers.Input(shape=(28,28,1))
x = tf.keras.layers.Conv2D(128, 7, strides=1, padding='same')(check_input)
x = tf.keras.layers.LeakyReLU()(x)
for _ in range(10):
  x = Residual_Block(128, 4)(x)
x = tf.keras.layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.Dense(10, activation='softmax')(x)

check_model = tf.keras.models.Model(check_input, x)

In [0]:
check_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [73]:
check_model.fit(x_train, y_train, epochs=10, batch_size = 128, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 249s 4ms/sample - loss: 0.1970 - acc: 0.9422 - val_loss: 4.8081 - val_acc: 0.1852
Epoch 2/10
60000/60000 [==============================] - 245s 4ms/sample - loss: 0.0631 - acc: 0.9805 - val_loss: 6.1955 - val_acc: 0.1135
Epoch 3/10
60000/60000 [==============================] - 246s 4ms/sample - loss: 0.0479 - acc: 0.9850 - val_loss: 4.5608 - val_acc: 0.1068
Epoch 4/10
60000/60000 [==============================] - 246s 4ms/sample - loss: 0.0387 - acc: 0.9876 - val_loss: 5.4629 - val_acc: 0.1135
Epoch 5/10
60000/60000 [==============================] - 245s 4ms/sample - loss: 0.0361 - acc: 0.9884 - val_loss: 4.4961 - val_acc: 0.1135
Epoch 6/10
41088/60000 [===================>..........] - ETA: 1:13 - loss: 0.0324 - acc: 0.9894

KeyboardInterrupt: ignored

```

class Residual_Block(tf.keras.layers.Layer):
  def __init__(self, filters, kernel_size, **kwargs):
    super(Residual_Block, self).__init__(**kwargs)
    self.filters = filters
    self.kernel_size = kernel_size
    self.conv1 = tf.keras.layers.Conv2D(self.filters, self.kernel_size, strides=1, padding='same')
    self.conv2 = tf.keras.layers.Conv2D(self.filters, self.kernel_size, strides=1, padding='same')
    self.bn = tf.keras.layers.BatchNormalization()
    self.ac = tf.keras.layers.LeakyReLU()

  def call(self, x):
    z = self.conv1(x)
    z = self.ac(z)
    z = self.bn(z)
    z = self.conv2(z)
    z = self.ac(z)
    z = self.bn(z)
    z = tf.keras.layers.Add()([x, z])
    return z
```

In [0]:
def residual_block_model():
  check_input = tf.keras.layers.Input(shape=(28,28,1))
  x = tf.keras.layers.Conv2D(128, 7, strides=1, padding='same')(check_input)

  z = x
  for _ in range(10):
    z = tf.keras.layers.Conv2D(128, 4, strides=1, padding='same')(z)
    z = tf.keras.layers.LeakyReLU()(z)
    z = tf.keras.layers.BatchNormalization()(z)
    z = tf.keras.layers.Conv2D(128, 4, strides=1, padding='same')(z)
    z = tf.keras.layers.LeakyReLU()(z)
    z = tf.keras.layers.BatchNormalization()(z)
    z = tf.keras.layers.Add()([x, z])
  x = tf.keras.layers.GlobalAvgPool2D()(z)
  x = tf.keras.layers.Dense(10, activation='softmax')(x)

  return tf.keras.models.Model(check_input, x)


In [0]:
model = residual_block_model()

In [92]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_196 (Conv2D)             (None, 28, 28, 128)  6400        input_15[0][0]                   
__________________________________________________________________________________________________
conv2d_197 (Conv2D)             (None, 28, 28, 128)  262272      conv2d_196[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_127 (LeakyReLU)     (None, 28, 28, 128)  0           conv2d_197[0][0]                 
___________________________________________________________________________________________

In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [94]:
model.fit(x_train, y_train, epochs=10, batch_size = 128, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 239s 4ms/sample - loss: 0.3159 - acc: 0.9027 - val_loss: 0.5942 - val_acc: 0.8487
Epoch 2/10
60000/60000 [==============================] - 243s 4ms/sample - loss: 0.0863 - acc: 0.9751 - val_loss: 0.3085 - val_acc: 0.9049
Epoch 3/10
60000/60000 [==============================] - 244s 4ms/sample - loss: 0.0632 - acc: 0.9811 - val_loss: 0.2328 - val_acc: 0.9365
Epoch 4/10
60000/60000 [==============================] - 243s 4ms/sample - loss: 0.0528 - acc: 0.9843 - val_loss: 0.7558 - val_acc: 0.8157
Epoch 5/10
60000/60000 [==============================] - 243s 4ms/sample - loss: 0.0495 - acc: 0.9850 - val_loss: 0.3084 - val_acc: 0.9116
Epoch 6/10
60000/60000 [==============================] - 242s 4ms/sample - loss: 0.0445 - acc: 0.9866 - val_loss: 0.1587 - val_acc: 0.9494
Epoch 7/10
60000/60000 [==============================] - 244s 4ms/sample - loss: 0.0401 - acc: 0.9877 - val_l

KeyboardInterrupt: ignored